In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

In [2]:
# parameters
customer_id = '137'
formatted_attribute = 'Width'
customer_name='%kimballmidwest%'
buckets = """Office Chairs"""

strategy_version_input=751
attribute_id_input=2425

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float, reg_ex,clean_data     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [5]:
# df = query_from_file(file_name='./query/uncurated.sql', params=params)
# df = df[df['bucket_id'].isin(buckets)]
# df=df[(df['external_id'].astype(str)!='200045713')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')]
# len(df)
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
# fields = ['custom_fields']
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
# df=dfs
print(len(df))
# reg_ex(df)

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']
df=df[(df['external_id'].astype(str)!='200045713')&(df['external_id'].astype(str)!='200045715')&(df['external_id'].astype(str)!='200045714')&(df['external_id'].astype(str)!='200046141')&(df['external_id'].astype(str)!='200046271')]

1876


In [6]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [7]:
print(len(df[(df['Width'].astype(str)!='[]')&(df['Width'].astype(str)!='nan')]))
df['Widths']=df['Width']
length_curation=df[(df['Widths'].astype(str)!='[]')&(df['Width'].astype(str)!='nan')]
length_curation['lengths']=length_curation['Widths'].astype(str)
clean_data(length_curation,'Widths','widths_final')
length_curation=curate(length_curation,'widths_final',formatted_attribute,'a-eghj-mo-su-z')

237
Repeated External IDs: []


In [8]:
print(len(df[(df['Width'].astype(str)=='[]')|(df['Width'].astype(str)=='nan')]))
length=df[(df['Width'].astype(str)=='[]')|(df['Width'].astype(str)=='nan')]
reg_ex(length)
length=length[['external_id','product_name','long_desc','width']]
print(len(length[length['width'].astype(str)!='[]']))
lengths=length[length['width'].astype(str)!='[]']
clean_data(lengths,'width','width_final')
curate(lengths,'width_final',formatted_attribute,'a-eghj-mo-su-z')
lengths[lengths['width_final'].astype(str)!='[]']

1639
13
Repeated External IDs: []


external_id                                   product_name  \
624       831356                          “Truck” First Aid Kit   
701       281818                      Quantum™ Twist Super Nova   
725        67347                 3-1/4" x 3-1/4" LED Work Light   
728       281003                                         281003   
780     839000NP               Yellow Chemical Absorbent Bundle   
781     839001NP            Gray Medium-Weight Absorbent Bundle   
782     839002NP        High Visibility Safety Absorbent Bundle   
783     839003NP  Pro•Clean Extra Heavy-Weight Absorbent Bundle   
784     839004NP             Compact Universal Absorbent Bundle   
867        69595   7" x 16" White Bubble Back West Coast Mirror   
868        69592   7" x 16" Black Bubble Back West Coast Mirror   
870        69582            7" x 16" Rib Back West Coast Mirror   
1468      831220                              Leather Bib Apron   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
624                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [9]:
print(len(length[length['Width'].astype(str)=='[]']))
length[length['Width'].astype(str)=='[]'][0:500]

KeyError: 'Width'

In [ ]:
err

# NA

In [ ]:
# regex_pattern_length=r'''(?i)(?<!cord.measures )(?<!capacity )((?<!\.)(?<!\d)(?<!\,)\d{1,3}(?!\d)(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:(?:")?(?:”)?(?:'{0,2})?(?:\Winc?h?e?s?)?(?:\Wft)?(?:\Wfoot)?)\WL(?:ength|ong)?\b)|((?:l(?:ength)?|long):\s?\d+\s?inc?h?e?s?)|(Le?n?g?t?h?\:\s?10\s?in)|((?<!\.)(?<!\/)\d+(?!\.).{0,3}Le?n?g?t?h?\b)|()'''
# df['length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

# regex_pattern_height=r'''(?i)((?<!\.)(?<!up.to.)(?<!length )(?<!\w)(?<!\/)(?<!\/)(?<!\.)\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\?(?:"?|”?|'{0,2}|\Winc?h?|\Wft|\Wfoot)\W(?:H(?:eight)?|tall)\b(?!.bulb)(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)(?!.?options))|(\bHe?i?g?h?t?\:\W?\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\\? (?:"|”|'{1,2}|\W?inc?h?|\Wft|\Wfoot))|(?:He?i?g?h?t?\:\s?\d+(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?\s?in)|()'''
# df['matches_height'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))

# regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Wi?d?t?h?\b)|()'
# df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))

# regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
# df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))

# regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)"
# df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))

# regex_pattern_blank=r'\d'
# df['new_matches'] = df['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

# regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
# df['blank_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df['blank_matches_long'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))    

In [ ]:
df=df[df['units'].astype(str)=='[]']

# Height

In [ ]:
heights=df[(df['length'].astype(str)!='[]')]
heights['matches_height']=heights['length'].apply(lambda x: natsorted(x))
# heights['height_final']=heights['matches_height'].apply(lambda x: re.sub(r'''(?i)\\{0,3}(?:''|"|”|\D?inc?h?e?s?)\\{0,3}x?a?\W?\\{0,3}\s?(?:inc?h?)?(?<!')\s?(?:l(?:ength)|l(?:ong)?)?(?:w(?:idth)|w(?:wide)?)?(?:h(?:eight)|(?:tall)?)?\:?\s?''',' in',str(x))).apply(lambda x: re.sub(r'(?i)[lwhd]\:\s?','',str(x)))#.apply(lambda x: re.sub(r'''(?i)(\\\"\s?L)|(\\?\"?\”?(?:inc?h?)?(?<!').(Length|L))|(\\?('')\W?(Length|L))|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(\\?(?:(?<!')'(?!'))?(?:f?t?)?(?:f?o?o?t?)?\W?(Length|L))|(square\W?foot)|(\'\\xa.{0,2}L)|\-foot''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x))).apply(lambda x: re.sub(r'''\\?\\?"\\?\\?xa\\?\\?\s?i?n?|inength|(?<=\d)\"long''',' in',str(x)))                                
clean_data(heights,'matches_height','matches_height')
heights=curate(heights, 'matches_height',formatted_attribute,'a-eghj-mo-su-z')
heights[curation_col]=heights[curation_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
heights[curation_col]=heights[curation_col].apply(lambda x: re.sub(r'(?i)\-foot in',' ft',str(x))).apply(lambda x: re.sub('ft\s\w\w','ft',str(x))).apply(lambda x: re.sub(r'\-','',str(x))).apply(lambda x: re.sub(r'ft.*"','ft"',str(x))).apply(lambda x: re.sub(r'  ',' ',str(x)))
match_first=heights[['external_id',curation_col]]
print(len(match_first))
rounding(heights, curation_col,'a-eghj-mo-su-z')

In [ ]:
del heights['height']
del heights['width']
del heights['diameter']
del heights['blank_matches']
del heights['blank_matches_long']
del heights['tripple_name']
del heights['tripple_long']
del heights['double_name']
del heights['double_long']
del heights['units']

# Heightz

In [ ]:
heightz=df[(df['height'].astype(str)!='[]')&(df['length'].astype(str)=='[]')]

# heightz['height_final']=heightz['height'].apply(lambda x: re.sub(r'''(?i)(\\?\"?\”?(?:inc?h?)?(?<!').(Height|H))|(\\?('')\W?(Height|H))|(\\" H)|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(?i)(\\?\\?"\\?\\?\s?xa\s?in)''',' in',str(x))).apply(lambda x: re.sub(r'''(\\?(?:(?<!')'(?!'))?(?:f?t?)?(?:f?o?o?t?)?\W?(Height|H))|(square\W?foot)|(\'\\xa.{0,2}H)''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)''',' in',str(x))).apply(lambda x: re.sub(r'.{0,3}\:\s?','',str(x))).apply(lambda x: re.sub(r"\[(?!')","['",str(x))).apply(lambda x: re.sub(r"\,(?!')",",'",str(x)))                                           
# heightz['height']=heights['matches_height'].apply(lambda x: natsorted(x))
clean_data(heightz,'height','height_final')
curate(heightz, 'height_final',formatted_attribute,'a-eghj-mo-su-z')
heightz[curation_col]=heightz[curation_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
heightz[curation_col]=heightz[curation_col].apply(lambda x: re.sub(r'2625','26.25',str(x)))
matchesheight=heightz[['external_id',curation_col]]
print(len(matchesheight))
rounding(heightz, curation_col,'a-eghj-mo-su-z')

# New

In [ ]:
# # from natsort import natsorted
# regex_pattern=r'(?i)((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!x\s)(?<!\d)\d+\.\d+[\W]{0,2}"(?!\sx)[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!x\s)(?<!\d)\d+.?\d+\/\d+[\W]{0,2}"(?!\sx)[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!x\s)(?<!\d)(?<!\.)(?<!\/)\d+(?!\.\d)[\W]{0,2}"(?!\sx)[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!x\s)(?<!\d)(?<!\.)(?<!\/)\d+(?!\.)[\W]{0,2}"(?!\sx)[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|(?i)(\d{1,3}(?:(?:\-?\s?\d+\/\d+)?(?:\.\d+)?)?.?(?!inche?s? wide)(?!inche?s? deep)(?!inche?s? tall)(?:(?:Inche?s?\s?l?o?n?g?)|(?:long)))|()'           
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

# # df['matches']=df['matches'].apply(lambda x: natsorted(x))

# regex_pattern_blank=r'\d'
# df['new_matches'] = df['length'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_new=df[(df['new_matches'].astype(str)!='[]')&(df['length'].astype(str)!='[]')]


# # df_new['matches']=df_new['matches'].apply(lambda x: natsorted(x))
# # df_new['length']=df_new['matches'].apply(lambda x: re.sub(r'''(?i)[^\d"”]{0,3}((?<!')'(?!'))[^\d"”]{0,4}(Length|L)|(\D{0,3}(foot|ft|feet))''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)[^\d']{0,3}(["”]).{0,4}(Length|L)|(\D{0,3}('')\W?(Length|L))|(\D{0,3}(inc?h?))''',' in',str(x)))                      
# # df_new['length']=df_new['length'].apply(lambda x: re.sub(r'(\\?\\?u200 in)','in',str(x))).apply(lambda x: re.sub(r'(\\?\\?u200 ft)','ft',str(x))).apply(lambda x: re.sub(r'(?i)[long]','',str(x))).apply(lambda x: re.sub(r'(?i)ies|i(?!\w)','in',str(x)))
# clean_data(df_new,'length','length')
# curate(df_new, 'length',formatted_attribute,'a-eghj-mo-su-z')
# print(len(df_new))
# # df_new[curation_col]=df_new['length']
# df_new[curation_col]=df_new[curation_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
# matchesnew=df_new[['external_id',curation_col]]
# rounding(df_new, 'length','a-eghj-mo-su-z')

# Max Min (Length is the max)

In [ ]:
pat_split=r'''(?i)(\d+.?\d*\/?\d*\W?(?:(?:I?n?c?h?"?)?(?:f?e?e?t?'?)?(?:f?o?o?t?'?)?)\W?x.?\d+\W?\d*\/?\d*\W?(?:(?:I?n?c?h?"?)?(?:f?e?e?t?'?)?(?:f?o?o?t?'?)?)\W)|()'''
df['matches_split'] = df['product_name'].apply(lambda x: re_extract(pat_split, str(x)))
regex_pattern_blank=r'(?i)(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df['matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
split=df[(df['matches_split'].astype(str)!='[]')&(df['length'].astype(str)=='[]')]
print(len(split))
# split.head()

In [ ]:
del split['buckets']
del split['width']
del split['diameter']
# del split['matches_ft']
# del split['new_matches']
# del split['na_measure_matches']

In [ ]:
split['matches_split']=split['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
split['matches_split']=split['matches_split'].str.get(0).apply(lambda x: x.lower())

In [ ]:
inch_foot=r'''(?i)(\d+.?(?:fe?e?t|foot|(?<!')'(?!'))\W?\d+\\{0,2}.?(?:inc?h?|"|''))|()'''
split['inch_foot'] = split['matches_split'].apply(lambda x: re_extract(inch_foot, str(x)))
split_inch_foot=split[split['inch_foot'].astype(str)!='[]']
split=split[split['inch_foot'].astype(str)=='[]']
split['inch_foot'].explode().value_counts()

In [ ]:
split_inch_foot['inch_foot']=split_inch_foot['inch_foot'].apply(lambda x: re.sub(r'''(\[\'|\\|\'\])''','',str(x)))
split_inch_foot

In [ ]:
def ft_inch(numbers):
    if str(numbers).find("'") > -1:
        try:
            nom = numbers.split("'")[0]
            den = numbers.split("'")[1]
            ft=int(nom)*12
            inch=den.replace(r'\"| ','')
            inch=int(inch)
            tmp = int(ft)+int(inch)
            return tmp
        except:
            return numbers
    else:
        return numbers

In [ ]:
x="2'6"
ft_inch(x)

In [ ]:
split_inch_foot['inch_foot']=split_inch_foot['inch_foot'].apply(lambda x: ft_inch(x))
split_inch_foot

In [ ]:
pat=r'''(?i)(\,)|()'''
split['doub'] = split['matches_split'].apply(lambda x: re_extract(pat, str(x)))
split=split[split['doub'].astype(str)=='[]']
print(len(split))

In [ ]:
split[['length', 'width']]=split['matches_split'].str.split("x",expand=True,)

In [ ]:
split['length']=split['length'].apply(lambda x: re.sub(r'\[\"','[',str(x))).apply(lambda x: re.sub(r'\"\]',']',str(x))).apply(lambda x: re.sub(r"\[\'",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|8x/1x|4x4 b|26x31 g|8x/1",'',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"2 24 in",'24 in',str(x))).apply(lambda x: re.sub(r"33 (?!in)",'33 in',str(x))).apply(lambda x: re.sub(r" ",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"5 inch",'31.5 in',str(x))).apply(lambda x: re.sub(r"20x24','20",'20 in',str(x)))                        
split['width']=split['width'].apply(lambda x: re.sub(r'\[\"','[',str(x))).apply(lambda x: re.sub(r'\"\]',']',str(x))).apply(lambda x: re.sub(r"\'\]",'',str(x))).apply(lambda x: re.sub(r"(?i)\W?feet|\W?foot|\\?\\?\'",'ft',str(x))).apply(lambda x: re.sub(r"ft\s?ft",'ft',str(x))).apply(lambda x: re.sub(r'-inch|-inc|\\?\\?\"',' in',str(x))).apply(lambda x: re.sub(r"\['|']|None| |4-f",'',str(x))).apply(lambda x: re.sub(r"in",' in',str(x))).apply(lambda x: re.sub(r"ft",' ft',str(x))).apply(lambda x: re.sub(r"24 in','20x24 in",'24 in',str(x))).apply(lambda x: re.sub(r"27b",'27 in',str(x))).apply(lambda x: re.sub(r"24(?! )",'24 in',str(x)))
print(len(split['width'].explode().value_counts()))
# split['width'].explode().value_counts()

In [ ]:
split=split[split['length'].astype(str)!=""]
split=split[split['width'].astype(str)!=""]

In [ ]:
split['length']=split['length'].apply(lambda x: re.sub(r'''1.?(?:fe?e?t|foot|(?<!')'(?!')).?2.?(?:inch|"|'')''','14 in',str(x))).apply(lambda x: re.sub(r'''2.?(?:fe?e?t|foot|(?<!')'(?!')).?6.?(?:inch|"|'')''','30 in',str(x)))
split['width']=split['width'].apply(lambda x: re.sub(r'''1.?(?:fe?e?t|foot|(?<!')'(?!')).?2.?(?:inch|"|'')''','14 in',str(x))).apply(lambda x: re.sub(r'''2.?(?:fe?e?t|foot|(?<!')'(?!')).?6.?(?:inch|"|'')''','30 in',str(x)))

In [ ]:
pat_split=r'''(?i)(ft)|()'''
split['ft'] = split['width'].apply(lambda x: re_extract(pat_split, str(x)))
split['ft_length'] = split['length'].apply(lambda x: re_extract(pat_split, str(x)))
print(len(split))

pat_split=r'''(?i)(in)|(\d\\{0,3}[\"\']|fo?o?e?e?t)|()'''
split['inch'] = split['matches_split'].apply(lambda x: re_extract(pat_split, str(x)))

In [ ]:
split_inch=split
pat_split=r'''(?i)(in)|()'''
split_inch=split_inch[split_inch['ft'].astype(str)!='[]']
split_inch['inch'] = split_inch['width'].apply(lambda x: re_extract(pat_split, str(x)))
split_inch=split_inch[split_inch['inch'].astype(str)!='[]']
split_inch['length']=split_inch['length'].apply(lambda x:re.sub(r'\s?in|\s?inch|ch|es|inche?s?|e|\)','',str(x))).apply(lambda x: re.sub(r'''1ft2''','14',str(x))).apply(lambda x: re.sub(r'1ft10','22',str(x))).apply(lambda x: re.sub(r'12132','32',str(x))).apply(lambda x: re.sub(r'\[|\]','',str(x)))
split_inch['width']=split_inch['width'].apply(lambda x:re.sub(r'\s?in|\s?inch|ch|es|inche?s?|e|\)','',str(x))).apply(lambda x: re.sub(r'''1ft2''','14',str(x))).apply(lambda x: re.sub(r'''2 ft6''','30',str(x))).apply(lambda x: re.sub(r'1 ft10','22',str(x))).apply(lambda x: re.sub(r'14 in','14',str(x))).apply(lambda x: re.sub(r'12132','32',str(x))).apply(lambda x: re.sub(r'\[|\]','',str(x)))     
split_inch['length']=split_inch['length'].astype(float)
split_inch['width']=split_inch['width'].astype(float)
split_inch['Max'] = split_inch[['length', 'width']].values.max(1)
split_inch['Min'] = split_inch[['length', 'width']].values.min(1)
split_inch['Max']=split_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
print(len(split_inch))
split_inch[curation_col]=split_inch['Max']
split_inch[curation_col]=split_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))


# split_ft=split
# pat_split=r'''(?i)(ft)|()'''
# split_ft['inch'] = split_ft['length'].apply(lambda x: re_extract(pat_split, str(x)))
# pat_split=r'''(?i)(ft)|()'''
# split_ft['ft'] = split_ft['width'].apply(lambda x: re_extract(pat_split, str(x)))
# split_ft=split_ft[(split_ft['ft'].astype(str)!='[]')&(split_ft['inch'].astype(str)!='[]')]
# split_ft['length']=split_ft['length'].apply(lambda x: re.sub(r'''14 in''','',str(x))).apply(lambda x:re.sub(r'\W?ft','',str(x)))
# split_ft['width']=split_ft['width'].apply(lambda x:re.sub(r'\W?ft','',str(x))).apply(lambda x: re.sub(r'','',str(x)))
# split_ft['length']=split_ft['length'].astype(float)
# split_ft['width']=split_ft['width'].astype(float)
# split_ft['Max'] = split_ft[['length', 'width']].values.max(1)
# split_ft['Min'] = split_ft[['length', 'width']].values.min(1)
# split_ft['Max']=split_ft['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# print(len(split_ft))

# split_ft[curation_col]=split_ft['Max']
# split_ft[curation_col]=split_ft[curation_col].apply(lambda x: str(f"['{x} ft']")).apply(lambda x: re.sub(r"'",'"',str(x)))
# rounding(split_inch,curation_col,'a-eghj-mo-su-z')

In [ ]:
msplit_inch=split_inch[['external_id',curation_col]]

# Unitless

In [ ]:
print(len(split[(split['inch'].astype(str)=='[]')&(split['ft'].astype(str)=='[]')&(split['ft_length'].astype(str)=='[]')]))
unitless=split[(split['inch'].astype(str)=='[]')&(split['ft'].astype(str)=='[]')&(split['ft_length'].astype(str)=='[]')]

# unitless['length']=unitless['length'].apply(lambda x:re.sub(r'(?i)\s?in|\s?inch|ch|inche?s?|e|\)|\/|t|2111|1213|2111','',str(x)))
# unitless['width']=unitless['width'].apply(lambda x:re.sub(r'(?i)\s?in|\s?inch|ch|inche?s?|e|\)|\/|t','',str(x)))

unitless['length']=unitless['length'].apply(lambda x: re.sub(r'\-','',str(x))).astype(float)
unitless['width']=unitless['width'].apply(lambda x: re.sub(r'\-','',str(x))).astype(float)
unitless['Max'] = unitless[['length', 'width']].values.max(1)
unitless['Min'] = unitless[['length', 'width']].values.min(1)
unitless['Max']=unitless['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
unitless[curation_col]=unitless['Max']
unitless[curation_col]=unitless[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))


unitless[curation_col]=unitless[curation_col].astype(str)
unitless[curation_col]=unitless[curation_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
unitless=unitless[unitless['external_id'].astype(str)!='200056446']
matchunitless=unitless[['external_id',curation_col]]
matchunitless[curation_col].explode().value_counts()
rounding(unitless,curation_col,'a-eghj-mo-su-z')

# N/A

In [ ]:
pat_na=r'''(?i)((?<!Cord Measures.{10})\d+\.\d+.{0,2}["'”].{0,2}[LWHD]?\b)|((?<!Cord Measures.{10})\d+.?\d+\/\d+.{0,2}["'”].{0,2}[LWHD]?\b)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.\d).{0,2}["'”].{0,2}[LWHD]?\b)|((?<!Cord Measures.{10})(?<!\.)(?<!\/)\d+(?!\.).{0,2}["'”].{0,2}[LWHD]?\b)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|(measure)|((?:(?:le?n?g?t?h?)|(?:He?i?g?h?t?)|(?:Wi?d?t?h?)|(?:De?p?t?h?))\:\s\d+\s?)|()'''
pat_measure=r'(?i)(measure)|()'
df_na=df

df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(pat_na, x))
df_na['na_measure_matches'] = df_na['long_desc'].apply(lambda x: re_extract(pat_measure, x))

df_na=df_na[(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')]
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
print(len(matchesna))

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchunitless))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(msplit_ft))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesheight))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesheight))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(match_first))

In [ ]:
print(len(df))
print(len(matchesna)+len(match_first))

In [ ]:
rounding(split_inch, curation_col,'a-eghj-mo-su-z')

In [ ]:
# rounding(df_new, curation_col,'a-eghj-mo-su-z')

In [ ]:
rounding(heights, curation_col,'a-eghj-mo-su-z')

In [ ]:
stop   

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, match_first)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Bed Bath & Beyond', attribute, df, matchesheight)    

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

looks_good('Bed Bath & Beyond', attribute, df, matchesna)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False)

looks_good('Bed Bath & Beyond', attribute, df, msplit_inch)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute}-{get_df_name(matches)}-matches.csv',index=False)

looks_good('Bed Bath & Beyond', attribute, df, matchunitless)